In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
search_results = pd.read_csv('../data/gp-search-20181012-181513.csv')
search_results.head()

,id,title,assignee,inventor/author,priority date,filing/creation date,publication date,grant date,result link,representative figure link
0,US-8271025-B2,Device network technology selection and displa...,At&T Mobility Ii Llc,"Arthur Brisebois, Sharat Chander, Yung Choi-Gr...",10/20/2008,10/20/2008,9/18/2012,9/18/2012,https://patents.google.com/patent/US8271025B2/en,https://patentimages.storage.googleapis.com/68...
1,US-6311175-B1,System and method for generating performance m...,Perot Systems Corp.,"Pieter Willem Adriaans, Arno Jan Knobbe, Marc ...",3/6/1998,3/6/1998,10/30/2001,10/30/2001,https://patents.google.com/patent/US6311175B1/en,https://patentimages.storage.googleapis.com/66...
2,US-5728098-A,Multi-angle bone screw assembly using shape-me...,"Sdgi Holdings, Inc.","Michael C. Sherman, Troy Drewry",11/7/1996,11/7/1996,3/17/1998,3/17/1998,https://patents.google.com/patent/US5728098A/en,https://patentimages.storage.googleapis.com/pa...
3,US-6675149-B1,Information technology project assessment meth...,International Business Machines Corporation,"Michael Ruffin, Joseph L. Temple, III, Carlos ...",11/2/1998,8/30/1999,1/6/2004,1/6/2004,https://patents.google.com/patent/US6675149B1/en,https://patentimages.storage.googleapis.com/bc...
4,US-6553263-B1,Implantable pulse generators using rechargeabl...,"Advanced Bionics Corporation, Quallion, Llc","Paul M. Meadows, Carla M. Mann, Hisashi Tsukam...",7/30/1999,7/28/2000,4/22/2003,4/22/2003,https://patents.google.com/patent/US6553263B1/en,https://patentimages.storage.googleapis.com/e5...


In [3]:
url = search_results.loc[0, 'result link']
r = requests.get(url)
soup = BeautifulSoup(r.content)
soup.find('h1').text

'US8271025B2 - Device network technology selection and display in multi-technology wireless environments \n        - Google Patents'

In [4]:
abstract = soup.find_all("div",  {"class": 'abstract'})
abstract[0].text.strip()

'System(s) and method(s) are provided to manage utilization of radio network technology and display thereof when multiple services and radio network technologies are available to a multi-technology mobile device. Management relies at least in part on a subscriber profile that comprises a network selection profile constructed through market policy, subscriber policy, and application policy for radio technology utilization. Network preference(s) profile is generated on per subscriber, or per subscriber type, basis and is conveyed to a subscriber station over the air. Initial subscriber profile can be delivered at a time of provisioning a multi-technology mobile device, and updated based at least upon subscriber demand, a schedule established by a network operator or service provider, or an event related to coverage area relocation or contracted services. Radio technology preferences and display of associated technologies available to a multi-technology mobile device can be dynamically co

In [5]:
def parse_abstract(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    try:
        abstract = soup.find_all('div', {'class': 'abstract'})[0].text.strip()
    except Exception as e:
        return None

In [6]:
from IPython import display

In [7]:
image_url = search_results.loc[0, 'representative figure link']
display.Image(url = image_url)

In [8]:
from tqdm import tqdm_notebook
from timeit import default_timer as timer

In [ ]:
titles = search_results['title']
urls = search_results['result link']

results = {}
start = timer()
for i, (title, url) in enumerate(zip(titles, urls)):
    print(f'{round(100 * (i / len(urls)), 2)}% complete.', end = '\r')
    results[title] = parse_abstract(url)
end = timer()

In [ ]:
from multiprocessing.dummy import Pool
import tqdm

pool = Pool(50)
r = tqdm_notebook(pool.imap_unordered(parse_abstract, urls), total = len(urls))
pool.close()
pool.join()

In [ ]:
soup.find("h1").text

In [ ]:
import json
with open('../data/found_tech_patents.json', 'w') as fout:
    # Iterate through each list element
    for l in results:
        # Write to file
        fout.write(json.dumps(l) + '\n')